In [70]:
import requests
import defs
import pandas as pd

In [71]:
session = requests.Session()

In [72]:
ins_df = pd.read_pickle("instruments.pkl")

In [73]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY','CHF', 'NZD', 'CAD']

In [74]:
ins_df.shape

(68, 5)

In [75]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = 'MBA'
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [76]:
code, res = fetch_candles('EUR_USD',10,"H1")

In [77]:
code

200

In [79]:
res

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 11025,
   'time': '2022-05-09T14:00:00.000000000Z',
   'bid': {'o': '1.05318', 'h': '1.05339', 'l': '1.05165', 'c': '1.05271'},
   'mid': {'o': '1.05326', 'h': '1.05346', 'l': '1.05172', 'c': '1.05278'},
   'ask': {'o': '1.05334', 'h': '1.05353', 'l': '1.05179', 'c': '1.05285'}},
  {'complete': True,
   'volume': 10461,
   'time': '2022-05-09T15:00:00.000000000Z',
   'bid': {'o': '1.05272', 'h': '1.05624', 'l': '1.05263', 'c': '1.05562'},
   'mid': {'o': '1.05278', 'h': '1.05632', 'l': '1.05270', 'c': '1.05570'},
   'ask': {'o': '1.05285', 'h': '1.05640', 'l': '1.05277', 'c': '1.05579'}},
  {'complete': True,
   'volume': 12981,
   'time': '2022-05-09T16:00:00.000000000Z',
   'bid': {'o': '1.05559', 'h': '1.05919', 'l': '1.05540', 'c': '1.05726'},
   'mid': {'o': '1.05568', 'h': '1.05926', 'l': '1.05548', 'c': '1.05735'},
   'ask': {'o': '1.05576', 'h': '1.05934', 'l': '1.05556', 'c': '1.05744'

In [59]:
def get_candles_df(json_response):
    
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']
    
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices: 
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [60]:
def save_file(candles_df,pair,granularity):
    candles_df.to_pickle(f'his_data/{pair}_{granularity}.pkl')

In [61]:
def create_data(pair, granularity):
    code, json_data = fetch_candles( pair, 4000, granularity)
    if code != 200:
        print(pair, "Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [62]:
for p1 in our_curr: 
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, 'H1')
            

EUR_USD loaded 3999 candles from 2021-09-17T06:00:00.000000000Z to 2022-05-09T22:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2021-09-17T03:00:00.000000000Z to 2022-05-09T22:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2021-09-17T06:00:00.000000000Z to 2022-05-09T22:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2021-09-17T06:00:00.000000000Z to 2022-05-09T22:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2021-09-17T06:00:00.000000000Z to 2022-05-09T22:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2021-09-17T06:00:00.000000000Z to 2022-05-09T22:00:00.000000000Z
USD_JPY loaded 3999 candles from 2021-09-17T06:00:00.000000000Z to 2022-05-09T22:00:00.000000000Z
USD_CHF loaded 3999 candles from 2021-09-17T01:00:00.000000000Z to 2022-05-09T22:00:00.000000000Z
USD_CAD loaded 3999 candles from 2021-09-17T06:00:00.000000000Z to 2022-05-09T22:00:00.000000000Z
GBP_USD loaded 3999 candles from 2021-09-17T06:00:00.000000000Z to 2022-05-09T22:00:00.000000000Z
GBP_JPY loaded 3999 

In [63]:
code, res = fetch_candles('EUR_USD',10,'H1' )

In [64]:
df = get_candles_df(res)

In [65]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-05-09T14:00:00.000000000Z,11025,1.05326,1.05346,1.05172,1.05278,1.05318,1.05339,1.05165,1.05271,1.05334,1.05353,1.05179,1.05285
1,2022-05-09T15:00:00.000000000Z,10461,1.05278,1.05632,1.05270,1.05570,1.05272,1.05624,1.05263,1.05562,1.05285,1.05640,1.05277,1.05579
2,2022-05-09T16:00:00.000000000Z,12981,1.05568,1.05926,1.05548,1.05735,1.05559,1.05919,1.05540,1.05726,1.05576,1.05934,1.05556,1.05744
3,2022-05-09T17:00:00.000000000Z,8394,1.05732,1.05753,1.05526,1.05584,1.05723,1.05746,1.05518,1.05574,1.05742,1.05762,1.05534,1.05593
4,2022-05-09T18:00:00.000000000Z,8703,1.05584,1.05669,1.05507,1.05660,1.05575,1.05662,1.05499,1.05652,1.05593,1.05677,1.05515,1.05668


In [66]:
ins_df.columns

Index(['name', 'type', 'displayName', 'pipLocation', 'marginRate'], dtype='object')

In [67]:
ins_df.name.unique()

array(['EUR_HUF', 'EUR_DKK', 'USD_MXN', 'GBP_USD', 'CAD_CHF', 'EUR_GBP',
       'GBP_CHF', 'USD_THB', 'USD_ZAR', 'EUR_NOK', 'GBP_PLN', 'EUR_HKD',
       'CHF_JPY', 'USD_SEK', 'GBP_SGD', 'GBP_HKD', 'EUR_NZD', 'SGD_CHF',
       'AUD_SGD', 'EUR_JPY', 'USD_CHF', 'USD_TRY', 'GBP_JPY', 'EUR_CZK',
       'EUR_TRY', 'USD_JPY', 'GBP_ZAR', 'SGD_JPY', 'USD_CZK', 'ZAR_JPY',
       'TRY_JPY', 'USD_DKK', 'EUR_PLN', 'AUD_CAD', 'USD_HKD', 'CAD_SGD',
       'GBP_NZD', 'CHF_ZAR', 'USD_CNH', 'NZD_USD', 'AUD_NZD', 'NZD_HKD',
       'USD_NOK', 'NZD_SGD', 'CAD_JPY', 'AUD_CHF', 'NZD_CHF', 'USD_SGD',
       'EUR_SEK', 'HKD_JPY', 'AUD_HKD', 'AUD_JPY', 'EUR_ZAR', 'EUR_AUD',
       'NZD_JPY', 'CHF_HKD', 'USD_CAD', 'EUR_CHF', 'EUR_CAD', 'USD_HUF',
       'NZD_CAD', 'EUR_SGD', 'AUD_USD', 'EUR_USD', 'GBP_AUD', 'USD_PLN',
       'CAD_HKD', 'GBP_CAD'], dtype=object)

In [68]:
eur_pk = pd.read_pickle("his_data/EUR_USD_H1.pkl")


In [69]:
eur_pk

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2021-09-17T06:00:00.000000000Z,3048,1.17704,1.17750,1.17688,1.17724,1.17697,1.17743,1.17681,1.17717,1.17711,1.17757,1.17695,1.17730
1,2021-09-17T07:00:00.000000000Z,4822,1.17722,1.17850,1.17716,1.17731,1.17716,1.17844,1.17709,1.17724,1.17729,1.17857,1.17722,1.17738
2,2021-09-17T08:00:00.000000000Z,4443,1.17730,1.17832,1.17684,1.17772,1.17723,1.17826,1.17677,1.17766,1.17737,1.17838,1.17690,1.17779
3,2021-09-17T09:00:00.000000000Z,6438,1.17774,1.17878,1.17740,1.17822,1.17768,1.17871,1.17734,1.17815,1.17780,1.17895,1.17746,1.17828
4,2021-09-17T10:00:00.000000000Z,4206,1.17820,1.17860,1.17787,1.17828,1.17814,1.17855,1.17781,1.17822,1.17827,1.17868,1.17793,1.17835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,2022-05-09T18:00:00.000000000Z,8703,1.05584,1.05669,1.05507,1.05660,1.05575,1.05662,1.05499,1.05652,1.05593,1.05677,1.05515,1.05668
3995,2022-05-09T19:00:00.000000000Z,9683,1.05661,1.05756,1.05560,1.05584,1.05654,1.05749,1.05552,1.05576,1.05668,1.05765,1.05569,1.05591
3996,2022-05-09T20:00:00.000000000Z,2018,1.05582,1.05624,1.05572,1.05602,1.05574,1.05615,1.05563,1.05592,1.05589,1.05633,1.05580,1.05613
3997,2022-05-09T21:00:00.000000000Z,1429,1.05594,1.05658,1.05544,1.05578,1.05544,1.05608,1.05501,1.05552,1.05644,1.05708,1.05559,1.05603
